In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('/home/joseph/Desktop/NNDL/Rainfall_data.csv')
df

,Year,Month,Day,Specific Humidity,Relative Humidity,Temperature,Precipitation
0,2000,1,1,8.06,48.25,23.93,0.00
1,2000,2,1,8.73,50.81,25.83,0.11
2,2000,3,1,8.48,42.88,26.68,0.01
3,2000,4,1,13.79,55.69,22.49,0.02
4,2000,5,1,17.40,70.88,19.07,271.14
...,...,...,...,...,...,...,...
247,2020,8,1,20.08,92.31,5.34,1203.09
248,2020,9,1,19.71,90.12,9.22,361.30
249,2020,10,1,18.43,82.69,12.62,180.18
250,2020,11,1,14.83,76.06,16.95,0.49


In [3]:
## setting date as index
df['DATE'] = pd.to_datetime(df[['Year', 'Month','Day']])
df = df.drop(columns=['Year', 'Month','Day'])
df.index = df["DATE"]
df.drop(columns=["DATE"],inplace = True)

In [4]:
# normalizing the dataset
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
df[['Specific Humidity', 'Relative Humidity', 'Temperature','Precipitation']] = scaler.fit_transform(df[['Specific Humidity', 'Relative Humidity', 'Temperature','Precipitation']])
df

,Specific Humidity,Relative Humidity,Temperature,Precipitation
DATE,,,,
2000-01-01,0.156440,0.235335,0.780171,0.000000
2000-02-01,0.201618,0.279764,0.857375,0.000084
2000-03-01,0.184761,0.142138,0.891914,0.000008
2000-04-01,0.542819,0.364457,0.721658,0.000015
2000-05-01,0.786244,0.628081,0.582690,0.207384
...,...,...,...,...
2020-08-01,0.966959,1.000000,0.024787,0.920195
2020-09-01,0.942009,0.961992,0.182446,0.276344
2020-10-01,0.855698,0.833044,0.320601,0.137812


In [5]:
#### training
train_data = df.iloc[:len(df)-12,:]
#### testing
test_data = df.iloc[len(df)-12:,:]

In [6]:
# split the data into x_train and y_train
x_train = train_data.iloc[:,1:]
y_train = train_data.iloc[:,0]
x_train


,Relative Humidity,Temperature,Precipitation
DATE,,,
2000-01-01,0.235335,0.780171,0.000000
2000-02-01,0.279764,0.857375,0.000084
2000-03-01,0.142138,0.891914,0.000008
2000-04-01,0.364457,0.721658,0.000015
2000-05-01,0.628081,0.582690,0.207384
...,...,...,...
2019-08-01,0.972926,0.058106,0.510115
2019-09-01,0.983860,0.080049,0.710470
2019-10-01,0.863415,0.292564,0.147320


In [7]:
# apply CNN on the data to predict the rainfall
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D
from keras.optimizers import *
from math import sqrt
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(3,1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='sgd', loss='mse')

# fit the model
model.fit(x_train, y_train, epochs=100, verbose=0)

# show the iteration for each epoch
for i in range(len(model.history.history['loss'])):
    print('>%d, l=%.3f' % (i+1, model.history.history['loss'][i]))

# evaluate the model
loss = model.evaluate(x_train, y_train, verbose=0)
print('MSE: %.3f, RMSE: %.3f' % (loss, sqrt(loss)))





2022-12-12 18:58:32.428325: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-12 18:58:32.568782: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-12 18:58:32.568805: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-12 18:58:33.177054: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

ImportError: cannot import name 'Mapping' from 'collections' (/usr/lib/python3.10/collections/__init__.py)

In [ ]:
# predict the rainfall based on the CNN model
x_test = test_data.iloc[:,1:]
y_test = test_data.iloc[:,0]
yhat = model.predict(x_test)
yhat

# plot the predicted rainfall and the actual rainfall
import matplotlib.pyplot as plt
plt.plot(yhat, label = 'predicted')
plt.plot(y_test.values, label = 'actual')
plt.legend()
plt.show()

# show classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, yhat))

# show confusion matrix
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, yhat))


In [ ]:
# show the accuracy score
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, yhat))
